In [ ]:
from google.colab import files
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor  
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.svm import NuSVR

# Polynomial regression 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

files.upload()


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Saving train_v9rqX0R.csv to train_v9rqX0R.csv


In [ ]:
df=pd.read_csv('train_v9rqX0R.csv')

# Encoding and droping rows that are not necessary 
lb= LabelEncoder()

df['item type']= lb.fit_transform(df['Item_Type'])
df['item fat content']= lb.fit_transform(df['Item_Fat_Content'])
df['outlet location type']= lb.fit_transform(df['Outlet_Location_Type'])
df['outlet type']= lb.fit_transform(df['Outlet_Type'])

# df.dropna(subset = ['Item_Weight'], inplace=True)

df['Outlet_Size'].fillna("No Size", inplace=True)
df['Item_Weight'].fillna(0, inplace=True)

df['Outlet Size']= lb.fit_transform(df['Outlet_Size'])

df=df.drop(columns=['Outlet_Size','Item_Type','Item_Fat_Content','Outlet_Location_Type','Outlet_Type'])

# Correlation Map and finding features that are correlated to the target
cor=df.corr()
plt.figure(figsize=(16,16))
sns.heatmap(cor, annot=True)

cor_target = abs(cor["Item_Outlet_Sales"])
print(cor_target[cor_target>0.1])

# df=df[['Item_MRP','outlet identifier','outlet location type','outlet type','Item_Outlet_Sales']]

# splitting into label and predictors

label=df['Item_Outlet_Sales']
df=df.loc[:,df.columns != 'Item_Outlet_Sales' ]
df

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,item type,item fat content,outlet location type,outlet type,Outlet Size
0,FDA15,9.300,0.016047,249.8092,OUT049,1999,4,1,0,1,1
1,DRC01,5.920,0.019278,48.2692,OUT018,2009,14,2,2,2,1
2,FDN15,17.500,0.016760,141.6180,OUT049,1999,10,1,0,1,1
3,FDX07,19.200,0.000000,182.0950,OUT010,1998,6,2,2,0,2
4,NCD19,8.930,0.000000,53.8614,OUT013,1987,9,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,0.056783,214.5218,OUT013,1987,13,1,2,1,0
8519,FDS36,8.380,0.046982,108.1570,OUT045,2002,0,2,1,1,2
8520,NCJ29,10.600,0.035186,85.1224,OUT035,2004,8,1,1,1,3
8521,FDN46,7.210,0.145221,103.1332,OUT018,2009,13,2,2,2,1


In [ ]:
x_train,x_test,y_train,y_test=train_test_split (df,label,test_size=0.05)
item_code =x_test[['Item_Identifier']]
Outlet_Identifier =x_test[['Outlet_Identifier']]

x_train=x_train.drop(columns=['Item_Identifier','Outlet_Identifier'])
x_test=x_test.drop(columns=['Item_Identifier','Outlet_Identifier'])

In [ ]:
#### Linear Regression

# lr=LinearRegression()
# lr.fit(x_train,y_train)

# y_predict=lr.predict(x_test)
# y_train_predict=lr.predict(x_train)

# print('Linear regression: \n')
# print(np.sqrt(mean_squared_error(y_train,y_train_predict)))
# print(np.sqrt(mean_squared_error(y_test,y_predict)))
# print( lr.score(x_train,y_train))



In [ ]:
#### Polynomial Regression
  
# poly = PolynomialFeatures(degree = 3) 
# X_poly = poly.fit_transform(x_train)

# poly.fit(X_poly,y_train) 
# lr2 = LinearRegression() 
# lr2.fit(X_poly,y_train) 

# y_predict_poly = lr2.predict(poly.fit_transform(x_test))
# y_train_predict_poly = lr2.predict(poly.fit_transform(x_train))
    

# print('\nPolynomial regression for i=' +str(3)+' \n')
# print(np.sqrt(mean_squared_error(y_train,y_train_predict_poly)))
# print(np.sqrt(mean_squared_error(y_test,y_predict_poly)))
# print( lr2.score(X_poly,y_train))

In [ ]:
#### Decision Tree Regressor

regressor = DecisionTreeRegressor(ccp_alpha=0.0, criterion='mae', max_depth=5,
                      max_features=9, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=4,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')
## Grid Search

# param_grid = {'max_depth': [5],'max_features': [9],
#               'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
#               'min_samples_split':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
#               'criterion': [ 'friedman_mse', 'mae', 'mse']}

# grid=GridSearchCV(regressor,param_grid)
# bp= grid.fit(x_train,y_train)

# print(bp.best_estimator_)

regressor.fit(x_train,y_train) 

y_predict=regressor.predict(x_test)
y_train_predict=regressor.predict(x_train)

print(np.sqrt(mean_squared_error(y_train,y_train_predict)))
print(np.sqrt(mean_squared_error(y_test,y_predict)))
print(regressor.score(x_test,y_test))

In [ ]:
### Nu Support Vector Regression

# nsvr=NuSVR(C=10, cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='linear',
#       max_iter=-1, nu=0.4, shrinking=True, tol=0.001, verbose=False)

# # param_grid = {'nu': [0.001, 0.005, 0.01, 0.02, 0.05, 0.1], 'C': [0.1, 0.2, 0.25, 0.5, 1, 1.5, 2], 
# #               'kernel':['poly', 'rbf'] }

# # grid=GridSearchCV(nsvr,param_grid)
# # bp= grid.fit(x_train,y_train)
# # print(bp.best_estimator_)

# nsvr.fit(x_train,y_train) 

# y_predict=nsvr.predict(x_test)
# y_train_predict=nsvr.predict(x_train)

# print(np.sqrt(mean_squared_error(y_train,y_train_predict)))
# print(np.sqrt(mean_squared_error(y_test,y_predict)))
# print(nsvr.score(x_test,y_test))

1204.552853339788
1167.30342012356
0.47902303919273115


In [ ]:
### XGB Regression

xg = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=8, missing=None, n_estimators=50,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0.01, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

# param_grid = { 'subsample':[i/100.0 for i in range(75,90,5)],'colsample_bytree':[i/100.0 for i in range(75,90,5)]}

# grid=GridSearchCV(xg,param_grid)
# bp= grid.fit(x_train,y_train)
# print(bp.best_estimator_)


xg.fit(x_train,y_train) 

y_predict=xg.predict(x_test)
y_train_predict=xg.predict(x_train)

print(np.sqrt(mean_squared_error(y_train,y_train_predict)))
print(np.sqrt(mean_squared_error(y_test,y_predict)))
print(xg.score(x_test,y_test))

991.8393798209565
1003.7839881909196
0.5913234243939323


In [ ]:
# Pre-processing the test set

files.upload()
df_test=pd.read_csv('test_AbJTz2l.csv')

lb= LabelEncoder()
df_test['item type']= lb.fit_transform(df_test['Item_Type'])
df_test['item fat content']= lb.fit_transform(df_test['Item_Fat_Content'])
df_test['outlet location type']= lb.fit_transform(df_test['Outlet_Location_Type'])
df_test['outlet type']= lb.fit_transform(df_test['Outlet_Type'])


df_test['Outlet_Size'].fillna("No Size", inplace=True)
df_test['Item_Weight'].fillna(0, inplace=True)

df_test['Outlet Size']= lb.fit_transform(df_test['Outlet_Size'])


Outlet_Identifier =df_test[['Outlet_Identifier']]
item_code=df_test[['Item_Identifier']]

df_test=df_test.drop(columns=['Item_Identifier','Outlet_Size','Item_Type','Item_Fat_Content','Outlet_Identifier','Outlet_Location_Type','Outlet_Type'])


y = lr2.predict(poly.fit_transform(df_test))
# y = xg.predict(df_test)
y=y.reshape(-1,1)


In [ ]:
df_final = pd.DataFrame()
df_final['Item_Identifier']=item_code['Item_Identifier']
df_final['Outlet_Identifier']=Outlet_Identifier['Outlet_Identifier']
df_final['Item_Outlet_Sales']=y

df_final['Item_Outlet_Sales']=df_final['Item_Outlet_Sales'].abs()

df_final.to_csv('final.csv')
